# Bikes Rental Dashboard

## team names :

### Omar Mohamed Abd El-Hamid G-10
### Hesham Tarek Morshdey G-10

## import LIbraries

In [1]:
import dash
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from logging import PlaceHolder
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import os
import warnings
import re
import numpy as np
import base64

## Exploratory Data Analysis and Preprocessing

In [2]:
df = pd.read_csv('bike-rental.csv')

In [3]:
df.head()

,ID,Date,y,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [4]:
new_cols = []
def edit_cols_names(cols):
    for col in cols:
        x = r"\w+\s\w+|\w+"
        s = re.findall(x,col)
        if len(s) > 1 and len(s[0]) > 2 and len(s[1]) > 3:
            res = s[0]+ " " + s[1]
            new_cols.append(res)
        else:
            res = s[0]
            new_cols.append(res)

In [5]:
edit_cols_names(df.columns)
df.columns = new_cols
df = df.rename(columns={'y': 'Total Bikes'})

In [6]:
df.head()

,ID,Date,Total Bikes,Hour,Temperature,Humidity,Wind speed,Visibility,Dew point temperature,Solar Radiation,Rainfall,Snowfall,Seasons,Holiday,Functioning Day
0,0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [7]:
df['Date']=pd.to_datetime(df['Date'],format="%d/%m/%Y")

In [8]:
df['Month'] = df['Date'].dt.month
df['WeekDay'] = df['Date'].dt.day_name()
df['Year'] = df['Date'].dt.year

In [9]:
df.head(5)

,ID,Date,Total Bikes,Hour,Temperature,Humidity,Wind speed,Visibility,Dew point temperature,Solar Radiation,Rainfall,Snowfall,Seasons,Holiday,Functioning Day,Month,WeekDay,Year
0,0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday,2017
1,1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday,2017
2,2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday,2017
3,3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday,2017
4,4,2017-12-01,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday,2017


In [10]:
years = df['Year'].unique()
months = df['Month'].unique()
hours = df['Hour'].unique()

In [11]:
df.columns

Index(['ID', 'Date', 'Total Bikes', 'Hour', 'Temperature', 'Humidity',
       'Wind speed', 'Visibility', 'Dew point temperature', 'Solar Radiation',
       'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning Day',
       'Month', 'WeekDay', 'Year'],
      dtype='object')

In [12]:
total_num_per_autumn = df.groupby('Seasons')['Total Bikes'].agg(np.sum)[0]
total_num_per_spring = df.groupby('Seasons')['Total Bikes'].agg(np.sum)[1]
total_num_per_summer = df.groupby('Seasons')['Total Bikes'].agg(np.sum)[2]
total_num_per_winter = df.groupby('Seasons')['Total Bikes'].agg(np.sum)[3]

## Dash App

In [13]:
app = JupyterDash(external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [14]:
colors = {

    'background': 'rgb(57,105,172)' ,
    'sec_background':'rgb(228,26,28)',
    'text': '#fff',
    'summer_text':'#3CA4FF',
    'spring_text':'#f44336',
    'winter_text':'#5A9E6F',
    'autumn_text': '#FCEB52'
    
}

divBorderStyle = {
    'backgroundColor' : '#393939',
    'borderRadius': '12px',
    'lineHeight': 0.9,
}

In [15]:
image_filename = 'seoul.jpg'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
encoded_image = encoded_image.decode('utf-8')

In [16]:
app.layout = html.Div(children=[
    
    html.Img(src='data:image/jpg;base64,{}'.format(encoded_image)),

    html.H1('Bikes Rental Visualization',
            style={'font_family': 'Helvetica Neue','color': colors['text'],'backgroundColor': colors['background']
                   ,'textAlign':'center'}),
  
    html.Br(), 
 
    html.Div([
            html.Div([
                html.H4(children='Total # bikes per Summer',
                       style={
                           'textAlign': 'center',
                           'color': colors['summer_text'],
                       }
                       ),
                html.P(f"{total_num_per_summer}",
                       style={
                    'textAlign': 'center',
                    'color': colors['summer_text'],
                    'fontSize': 30,
                }
                ),
        
            ],
                style=divBorderStyle,
                className='three columns',
            ),
            html.Div([
                html.H4(children='Total # bikes per Winter',
                       style={
                           'textAlign': 'center',
                           'color': colors['winter_text'],
                       }
                       ),
                html.P(f"{total_num_per_winter}",
                       style={
                    'textAlign': 'center',
                    'color': colors['winter_text'],
                    'fontSize': 30,
                }
                ),
                
            ],
                style=divBorderStyle,
                className='three columns'),
        
            html.Div([
                html.H4(children='Total # bikes per Spring',
                       style={
                           'textAlign': 'center',
                           'color': colors['spring_text'],
                       }
                       ),
                html.P(f"{total_num_per_spring}",
                       style={
                    'textAlign': 'center',
                    'color': colors['spring_text'],
                    'fontSize': 30,
                }
                ),
                
            ],
                style=divBorderStyle,
                className='three columns')
        ,
        
             html.Div([
                html.H4(children='Total # bikes per Autumn',
                       style={
                           'textAlign': 'center',
                           'color': colors['autumn_text'],
                       }
                       ),
                html.P(f"{total_num_per_autumn}",
                       style={
                    'textAlign': 'center',
                    'color': colors['autumn_text'],
                    'fontSize': 30,
                }
                ),
                
            ],
                style=divBorderStyle,
                className='three columns')
        
        ], className='row')
    ,
    html.Br(),
    html.H2('Bikes Rental Stats',
            style={'width': '30%','height': '20%','border':'1px solid', 'border-radius': 10
                   ,'color': colors['text'],'backgroundColor': colors['sec_background']
                   ,'textAlign':'center','display': 'inline-block'}),
    html.Div([
        html.Div([
            html.Pre(children="Select Year", style={"fontSize":"150%"}),
            dcc.Dropdown(
                    id="dropdown",
                    options=[{"label": x, "value": x} for x in years],
                    value=years[0],
                    clearable=False,
                    )
                ], className='six columns'),

    ], className='row'),
    dcc.Graph(id="bar-chart", figure={}),
    html.Br(),


    html.H2('Rush Hours',style={'width': '30%','height': '20%','border':'1px solid', 'border-radius': 10,
                                       'color': colors['text'],'backgroundColor': colors['sec_background']
                                       ,'textAlign':'center','display': 'inline-block'}),
    
    
    html.Div([
            dcc.Graph(id = 'Hoilday_graph', figure = {})
        ]),
    
    html.Br(),
    
    html.H2('Total Bikes Per Seasons',style={'width': '30%','height': '20%','border':'1px solid', 'border-radius': 10,
                                       'color': colors['text'],'backgroundColor': colors['sec_background']
                                       ,'textAlign':'center','display': 'inline-block'}),
    html.Div([
            dcc.Graph(id = 'graph3', figure = {})
        ])
    
])

In [17]:
@app.callback(
    Output("bar-chart", "figure"), 
    Output('Hoilday_graph','figure'),
    Output('graph3','figure'),
    [Input("dropdown", "value")])
def update_bar_chart(year):
  
    df_update = df[df['Year']==year]
    
    y_update = df_update.groupby('Seasons')['Total Bikes'].agg(np.sum)
   
    y_update = np.array(y_update)
    
    seasons = np.array(df_update['Seasons'].unique())
    
    result1_fig = px.bar(df_update , x = seasons, y = y_update , labels=dict(x='Season' , y='Total number of bikes'))
    
    result2_fig= px.bar(df , x='Hour' , y ='Total Bikes' , barmode='group' , color='Holiday',labels=dict(y='total number of bikes'))

    result3_fig = px.sunburst(df , color='Total Bikes' , values='Total Bikes' 
            , path=['Seasons' , 'Holiday'] , hover_name='Seasons' , height=600)
    
    return result1_fig, result2_fig, result3_fig


In [18]:
app.run_server(debug=True , use_reloader=False)

Dash app running on http://127.0.0.1:8050/
